In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
!pip install catboost

     |████████████████████████████████| 69.2MB 43kB/s 


In [2]:
data = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
testt = test.copy()
data['Market Value'] = data['Market Value'].astype('float64')
test['Market Value'] = test['Market Value'].astype('float64')

data = data.fillna(0)
test = test.fillna(0)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3199 entries, 0 to 3198
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Market Value                3199 non-null   float64
 1   size(sqm)                   3199 non-null   float64
 2   NoBed                       3199 non-null   int64  
 3   Notoilet                    3199 non-null   int64  
 4   ppty type                   3199 non-null   int64  
 5   No of floors                3199 non-null   int64  
 6   No of buildings             3199 non-null   int64  
 7   Boys Quarters               3199 non-null   int64  
 8   car park                    3199 non-null   int64  
 9   age of property             3199 non-null   int64  
 10  security                    3199 non-null   int64  
 11  location                    3199 non-null   int64  
 12  Condition                   3199 non-null   int64  
 13  availability of facilities  3199 

In [4]:

df = pd.concat([data, test]).reset_index(drop = True)
df = pd.get_dummies(df,columns = ['location'],prefix_sep = '-',drop_first=True)
data = df.iloc[:3199]
test  = df.iloc[3199:]

In [5]:
test

,Market Value,size(sqm),NoBed,Notoilet,ppty type,No of floors,No of buildings,Boys Quarters,car park,age of property,security,Condition,availability of facilities,proximity,finishes,ceiling,windows,painting,Roof,location-2,location-3,location-4,location-5,location-6,location-7,location-8,location-9,location-10
3199,16180000.0,809.00,3,3,2,2,2,1,1,9,3,2,3,1,1,1,2,1.0,2,0,0,0,1,0,0,0,0,0
3200,21000000.0,1050.00,3,3,3,2,2,2,1,8,3,3,2,1,1,1,2,1.0,2,0,0,1,0,0,0,0,0,0
3201,28522500.0,570.45,3,3,2,1,2,2,1,9,2,2,3,2,1,1,2,1.0,2,0,0,0,0,0,0,0,1,0
3202,45000000.0,750.00,3,3,1,1,2,2,1,15,2,2,2,1,1,1,2,1.0,2,0,1,0,0,0,0,0,0,0
3203,16500000.0,550.00,3,3,2,2,3,2,1,9,2,3,3,1,1,3,2,1.0,2,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3994,19600000.0,980.00,3,4,2,2,3,2,1,8,3,3,3,1,1,3,2,1.0,2,0,0,0,0,0,0,1,0,0
3995,11100000.0,555.00,3,3,2,2,1,2,1,7,3,2,3,1,1,1,2,1.0,2,0,0,0,0,0,0,1,0,0
3996,17100000.0,855.00,4,3,3,2,2,2,1,7,3,3,3,1,1,3,2,1.0,2,0,0,0,0,0,0,1,0,0
3997,16400000.0,820.00,3,3,1,2,1,2,1,4,3,3,2,1,1,1,2,1.0,2,0,0,1,0,0,0,0,0,0


In [6]:
from sklearn.model_selection import train_test_split
X = data.drop(['Market Value'],axis = 1)
test_pred = test.drop(['Market Value'],axis = 1)
#X = data[['size(sqm)','location', 'Condition','ppty type']]
y = np.log(data['Market Value'].astype('float64'))

In [7]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=30)

In [8]:
X_test

,size(sqm),NoBed,Notoilet,ppty type,No of floors,No of buildings,Boys Quarters,car park,age of property,security,Condition,availability of facilities,proximity,finishes,ceiling,windows,painting,Roof,location-2,location-3,location-4,location-5,location-6,location-7,location-8,location-9,location-10
2175,256.51,3,3,5,2,1,2,1,10,2,3,2,1,1,1,2,1.0,2,0,0,0,0,0,0,0,0,0
358,650.00,3,3,3,2,2,2,1,8,2,2,3,1,1,1,2,1.0,2,0,1,0,0,0,0,0,0,0
2928,570.00,3,3,1,1,2,2,1,3,3,3,2,1,1,1,2,1.0,2,0,0,0,0,0,0,1,0,0
2969,980.00,3,3,2,2,3,2,1,5,3,2,3,1,1,1,2,1.0,2,0,0,1,0,0,0,0,0,0
1778,569.00,3,3,1,2,3,2,1,5,2,2,3,1,1,3,2,1.0,2,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,1050.00,3,3,2,2,2,2,1,4,3,3,2,1,2,1,2,1.0,2,0,1,0,0,0,0,0,0,0
857,544.00,3,3,3,2,3,2,1,13,2,2,3,1,1,3,2,1.0,2,0,0,0,0,1,0,0,0,0
2404,900.00,3,3,1,2,2,2,1,16,2,3,3,1,1,1,2,1.0,2,0,0,0,0,0,1,0,0,0
2553,465.00,3,4,2,2,2,2,1,8,1,3,3,1,1,1,2,1.0,2,0,0,0,0,0,1,0,0,0


In [9]:
# import catboost as cbt
# cbt = cbt.CatBoostRegressor(random_state = 34,n_estimators =5000)
# cbt.fit(X_train, y_train)

# # Make predictions
# predscbt = cbt.predict(X_test)

# # Score
# print(mean_squared_error(np.exp(y_test), np.exp(predscbt))**0.5)

In [10]:
import sklearn.metrics as metrics
def regression_results(y_true, y_pred):
    
    # Regression metrics
    explained_variance=metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_true, y_pred) 
    mse=metrics.mean_squared_error(y_true, y_pred) 
    mean_squared_log_error=metrics.mean_squared_log_error(y_true, y_pred)
    median_absolute_error=metrics.median_absolute_error(y_true, y_pred)
    r2=metrics.r2_score(y_true, y_pred)

    print('explained_variance: ', round(explained_variance,4))    
    print('mean_squared_log_error: ', round(mean_squared_log_error,4))
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))
    result = [round(explained_variance,4), round(mean_squared_log_error,4),round(r2,4),round(mean_absolute_error,4),round(mse,4),round(np.sqrt(mse),4)]
    return result

In [11]:
import catboost as cbt
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
cval_err = 0
ccp = 0
cbt = cbt.CatBoostRegressor(random_state = 34,n_estimators =3000)
for train, val in kf.split(data):
    
    cbt.fit(X_train,y_train)
    cp = cbt.predict(X_test)
    cy = cbt.predict(X_train)
    cPredictions = cbt.predict(test_pred)
    ce = np.exp(cp)-np.exp(y_test)
    cval_err += np.sqrt(np.dot(ce,ce)/len(y_test))
crmse_10cv = cval_err/10
print(crmse_10cv)

Streaming output truncated to the last 5000 lines.
1002:	learn: 0.1324976	total: 1.36s	remaining: 2.7s
1003:	learn: 0.1323068	total: 1.36s	remaining: 2.7s
1004:	learn: 0.1322088	total: 1.36s	remaining: 2.7s
1005:	learn: 0.1321641	total: 1.36s	remaining: 2.7s
1006:	learn: 0.1320126	total: 1.36s	remaining: 2.7s
1007:	learn: 0.1319204	total: 1.36s	remaining: 2.69s
1008:	learn: 0.1318439	total: 1.36s	remaining: 2.69s
1009:	learn: 0.1317042	total: 1.36s	remaining: 2.69s
1010:	learn: 0.1315734	total: 1.37s	remaining: 2.69s
1011:	learn: 0.1314345	total: 1.37s	remaining: 2.69s
1012:	learn: 0.1313925	total: 1.37s	remaining: 2.69s
1013:	learn: 0.1313146	total: 1.37s	remaining: 2.68s
1014:	learn: 0.1312872	total: 1.37s	remaining: 2.68s
1015:	learn: 0.1312199	total: 1.37s	remaining: 2.68s
1016:	learn: 0.1311917	total: 1.37s	remaining: 2.68s
1017:	learn: 0.1309281	total: 1.38s	remaining: 2.68s
1018:	learn: 0.1308574	total: 1.38s	remaining: 2.68s
1019:	learn: 0.1307924	total: 1.38s	remaining: 2.67s


In [12]:
import xgboost as xg
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
xval_err = 0
xgb = xg.XGBRegressor(random_state = 34,n_estimators = 3000)
for train, valid in kf.split(data):
    
    xgb.fit(X_train,y_train)
    xp = xgb.predict(X_test)
    xy = xgb.predict(X_train)
    xPredictions = xgb.predict(test_pred)
    xe = np.exp(xp)-np.exp(y_test)
    xval_err += np.sqrt(np.dot(xe,xe)/len(y_test))
xrmse_10cv = xval_err/10
print(xrmse_10cv)

[22:27:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:27:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:27:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:27:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:27:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
4185349.8071758547


In [13]:
import lightgbm as lg
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
lval_err = 0
lgb = lg.LGBMRegressor(random_state = 34,n_estimators =3000)
for train, val in kf.split(data):
    
    lgb.fit(X_train,y_train)
    lp = lgb.predict(X_test)
    ly = lgb.predict(X_train)
    lPredictions = lgb.predict(test_pred)
    le = np.exp(lp)-np.exp(y_test)
    lval_err += np.sqrt(np.dot(le,le)/len(y_test))
lrmse_10cv = lval_err/10
print(lrmse_10cv)

4417286.0293780565


In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
lrval_err = 0
lr = LinearRegression()

for train, val in kf.split(data):
    
    lr.fit(X_train,y_train)
    lrp = lr.predict(X_test)
    lry = lr.predict(X_train)
    lrPrediction = lr.predict(test_pred)
    lre = np.exp(lrp)-np.exp(y_test)
    lrval_err += np.sqrt(np.dot(lre,lre)/len(y_test))
lrrmse_10cv = lrval_err/10
print(lrrmse_10cv)

9329558.023832595


In [15]:
from sklearn.svm import SVR
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
sval_err = 0
np.random.seed(42)
svr = SVR()
for train, val in kf.split(data):
    
    svr.fit(X_train,y_train)
    sp = svr.predict(X_test)
    sy = svr.predict(X_train)
    sPredictions = svr.predict(test_pred)
    se = np.exp(sp)-np.exp(y_test)
    sval_err += np.sqrt(np.dot(se,se)/len(y_test))
srmse_10cv = sval_err/10
print(srmse_10cv)

11249753.845894797


In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
rval_err = 0
rfr = RandomForestRegressor(random_state = 34,n_estimators =3000)
for train, test in kf.split(data):
    
    rfr.fit(X_train,y_train)
    rp = rfr.predict(X_test)
    ry = rfr.predict(X_train)
    rPredictions = rfr.predict(test_pred)
    re = np.exp(rp)-np.exp(y_test)
    rval_err += np.sqrt(np.dot(re,re)/len(y_test))
rrmse_10cv = rval_err/10
print(rrmse_10cv)

3884621.18220305


In [17]:
compile = pd.DataFrame({'Actual Value':testt['Market Value'],'CatBoost':np.exp(cPredictions),'XGBoost':np.exp(xPredictions),'LGBM':np.exp(lPredictions),'Linear Regression':np.exp(lrPrediction),'SVM':np.exp(sPredictions),'Random Forest':np.exp(rPredictions)})

In [18]:
compile_valid = pd.DataFrame({'Actual Value':y_test,'CatBoost':np.exp(cp),'XGBoost':np.exp(xp),'LGBM':np.exp(lp),'Linear Regression':np.exp(lrp),'SVM':np.exp(sp),'Random Forest':np.exp(rp)})

In [19]:
compile_train = pd.DataFrame({'Actual Value':y_train,'CatBoost':np.exp(cy),'XGBoost':np.exp(xy),'LGBM':np.exp(ly),'Linear Regression':np.exp(lry),'SVM':np.exp(sy),'Random Forest':np.exp(ry)})

In [20]:
compile.columns

Index(['Actual Value', 'CatBoost', 'XGBoost', 'LGBM', 'Linear Regression',
       'SVM', 'Random Forest'],
      dtype='object')

In [21]:
all_result = []
for i in ['CatBoost', 'XGBoost', 'LGBM', 'Linear Regression','SVM', 'Random Forest']:
  all_result.append(regression_results(compile['Actual Value'],compile[i]))

explained_variance:  0.8686
mean_squared_log_error:  0.0352
r2:  0.8666
MAE:  2299421.1411
MSE:  62729514588561.72
RMSE:  7920196.6256
explained_variance:  0.8155
mean_squared_log_error:  0.0537
r2:  0.8149
MAE:  2478640.3238
MSE:  87053665126411.12
RMSE:  9330255.3623
explained_variance:  0.8363
mean_squared_log_error:  0.0533
r2:  0.8353
MAE:  3170172.2476
MSE:  77473709592686.31
RMSE:  8801915.1094
explained_variance:  0.2876
mean_squared_log_error:  0.268
r2:  0.2622
MAE:  11834234.78
MSE:  347038214420441.06
RMSE:  18628961.7107
explained_variance:  0.0206
mean_squared_log_error:  0.4485
r2:  -0.0921
MAE:  16909396.5579
MSE:  513718127314411.75
RMSE:  22665350.8094
explained_variance:  0.8441
mean_squared_log_error:  0.0406
r2:  0.8412
MAE:  2321925.4329
MSE:  74691646618431.14
RMSE:  8642432.9108


In [22]:
all_result_valid = []
for i in ['CatBoost', 'XGBoost', 'LGBM', 'Linear Regression','SVM', 'Random Forest']:
  all_result_valid.append(regression_results(compile_valid['Actual Value'],compile_valid[i]))

explained_variance:  -858158420706564.8
mean_squared_log_error:  201.202
r2:  -3037587868432321.0
MAE:  31402658.7507
MSE:  1374418127656192.0
RMSE:  37073145.6402
explained_variance:  -933582913935172.4
mean_squared_log_error:  201.204
r2:  -3155799879596968.0
MAE:  31709415.8784
MSE:  1427905545333837.0
RMSE:  37787637.467
explained_variance:  -839395265277819.5
mean_squared_log_error:  201.205
r2:  -3008798517365286.0
MAE:  31330343.4615
MSE:  1361391802919646.5
RMSE:  36897043.2815
explained_variance:  -352351257833023.2
mean_squared_log_error:  201.0211
r2:  -2139735946152237.0
MAE:  28438326.0442
MSE:  968166848225853.5
RMSE:  31115379.6092
explained_variance:  -24664648890914.176
mean_squared_log_error:  199.5966
r2:  -1386371563716485.2
MAE:  24822003.5784
MSE:  627291881377703.8
RMSE:  25045795.6827
explained_variance:  -816625850392096.2
mean_squared_log_error:  201.0854
r2:  -2950487750557027.5
MAE:  31072640.9756
MSE:  1335007915963920.2
RMSE:  36537760.1388


In [23]:
all_result_train = []
for i in ['CatBoost', 'XGBoost', 'LGBM', 'Linear Regression','SVM', 'Random Forest']:
  all_result_train.append(regression_results(compile_train['Actual Value'],compile_train[i]))

explained_variance:  -946245172563556.0
mean_squared_log_error:  200.6905
r2:  -3145626679774264.5
MAE:  31119773.1226
MSE:  1385094659587778.5
RMSE:  37216859.8835
explained_variance:  -1024729300240566.6
mean_squared_log_error:  200.7038
r2:  -3262237490921407.0
MAE:  31388347.6754
MSE:  1436441188662136.8
RMSE:  37900411.4577
explained_variance:  -1030136371072379.1
mean_squared_log_error:  200.705
r2:  -3270499735774990.5
MAE:  31408367.8532
MSE:  1440079252675418.2
RMSE:  37948376.1533
explained_variance:  -351828346943576.3
mean_squared_log_error:  200.489
r2:  -2113288573429034.5
MAE:  27849830.6232
MSE:  930531501415954.9
RMSE:  30504614.4282
explained_variance:  -25134777846390.19
mean_squared_log_error:  199.5536
r2:  -1417058906926651.2
MAE:  24756766.4293
MSE:  623964927855409.5
RMSE:  24979289.979
explained_variance:  -899181249364769.0
mean_squared_log_error:  200.6567
r2:  -3065415350094865.0
MAE:  30884376.0397
MSE:  1349775692753054.2
RMSE:  36739293.5799


In [26]:
reg = pd.DataFrame({'Metrics':['Explained Variance','Mean Squared Log Error','R2','MAE','MSE','RMSE',],'CatBoost':all_result[0],'XGBoost':all_result[1],'LGBM':all_result[2],'Linear Regression':all_result[3],'SVM':all_result[4],'Random Forest':all_result[5]})

In [27]:
reg_valid = pd.DataFrame({'Metrics':['Explained Variance','Mean Squared Log Error','R2','MAE','MSE','RMSE',],'CatBoost':all_result_valid[0],'XGBoost':all_result_valid[1],'LGBM':all_result_valid[2],'Linear Regression':all_result_valid[3],'SVM':all_result_valid[4],'Random Forest':all_result_valid[5]})

In [28]:
reg_train = pd.DataFrame({'Metrics':['Explained Variance','Mean Squared Log Error','R2','MAE','MSE','RMSE',],'CatBoost':all_result_train[0],'XGBoost':all_result_train[1],'LGBM':all_result_train[2],'Linear Regression':all_result_train[3],'SVM':all_result_train[4],'Random Forest':all_result_train[5]})

In [29]:
compile.to_csv('Test ML Predictions.csv',index = False)
reg.to_csv('Test Regression Metrics Result.csv',index = False)

In [30]:
compile_valid.to_csv('Validation ML Predictions.csv',index = False)
reg_valid.to_csv('Validation Regression Metrics Result.csv',index = False)

In [31]:
compile_train.to_csv('Train ML Predictions.csv',index = False)
reg_train.to_csv('Train Regression Metrics Result.csv',index = False)